In [3]:
import os
import json
import pandas as pd
from chaino.rpc import RPC

## Load BSC transactions

The first stage of this snapshot used an archive of the Binance Smart Chain, which was queried from the official Binance RPC servers.

The BSC archive contains blocks ranging from 28904155 to 29984961 (2023-06-07 to 2023-07-15).

In [4]:
bsc_txs_filename = os.path.expanduser("../var/bsc-txs.csv.gz")
txs = pd.read_csv(bsc_txs_filename)
txs.head()

,block_id,tx_hash,method,from,to,quantity
0,28904155,0xec988543685dc345007c0562845dd1ed015d8312f0ca...,0xa9059cbb,0xe2fc31F816A9b94326492132018C3aEcC4a93aE1,0xAD29AbB318791D579433D831ed122aFeAf29dcfe,0
1,28904155,0xf86a744f01f6f32a84a1243317d03e32ef23df04c6b5...,0xa9059cbb,0xa180Fe01B906A1bE37BE6c534a3300785b20d947,0x55d398326f99059fF775485246999027B3197955,0
2,28904155,0x613ebd5603a636f8e432c57c10986c5242aff623ab86...,0xa9059cbb,0xdccF3B77dA55107280bd850ea519DF3705D1a75a,0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56,0
3,28904155,0xa83cb64ed72df1108d9cde2a1e9ffd89b3af75d864c8...,0xa9059cbb,0xEB2d2F1b8c558a40207669291Fda468E50c8A0bB,0x55d398326f99059fF775485246999027B3197955,0
4,28904155,0xfa6e2c53384f41d5b3a0299855a517e31267013af931...,0xa9059cbb,0x8894E0a0c962CB723c1976a4421c95949bE2D4E3,0x09E889BB4D5b474f561db0491C38702F367A4e4d,0


## Identify interacting wallet addresses

This snapshot begins by identifying all the addresses that ever interacted with Morphex contracts.
This list of addresses will be used in the next stage to query balances on-chain.

In [5]:
mpx_erc20_address = "0x94C6B279b5df54b335aE51866d6E2A56BF5Ef9b7"
fsmlp_erc20_address = "0x4e0e48b787E308049d0CA6bfAA84D5c61c5a4A1e"
thena_mpx_wbnb_address = "0x51BfC6e47c96d2b8c564B0DdD2C44fC03707cdc7" # LP ERC-20
thena_mpx_wbnb_gauge = "0x0d739cE843d0584aAE800f54685d1fa69cEC1190" # GaugeV2

### MPX ERC-20

In [6]:
mpx_txs = txs.query(f'to == "{mpx_erc20_address}"')
len(mpx_txs)

718

In [7]:
unique_mpx_addresses = set(mpx_txs['from'])
len(unique_mpx_addresses)

251

### fsMLP ERC-20

In [8]:
fsmlp_txs = txs.query(f'to == "{fsmlp_erc20_address}"')
len(fsmlp_txs)

7

In [9]:
unique_fsmlp_addresses = set(fsmlp_txs['from'])
len(unique_fsmlp_addresses)

1

### Thena MPX-WBNB LP

In [10]:
thena_txs = txs.query(f'to == "{thena_mpx_wbnb_address}"')
len(thena_txs)

235

In [11]:
unique_thena_addresses = set(thena_txs['from'])
len(unique_thena_addresses)

86

### Thena MPX-WBNB Gauge

In [12]:
thena_gauge_txs = txs.query(f'to == "{thena_mpx_wbnb_gauge}"')
len(thena_gauge_txs)

613

In [13]:
unique_thena_gauge_addresses = set(thena_gauge_txs['from'])
len(unique_thena_gauge_addresses)

86

### Combine unique addresses

In [16]:
addresses = unique_mpx_addresses | unique_thena_addresses | unique_thena_gauge_addresses | unique_fsmlp_addresses
addresses = list(addresses)
addresses.append(thena_mpx_wbnb_address) # also get balances for Thena LP
addresses.append(thena_mpx_wbnb_gauge) # also get balances for Thena LP
addresses = sorted(addresses)
with open("../data/bsc-addresses.json", "w") as f:
    json.dump(addresses, f, indent=2)
len(addresses)

253

## Call contracts on-chain to obtain balances

Binance RPCs  permit `eth_call` to be executed against a historical block.
For this snapshot, we queried contracts on-chain at block `29541500` (June 30, 2:20 UTC).

In [19]:
def parse_address(item):
    return item.split(",")[2].split("'")[1]

### MPX ERC-20

Obtain balances for all interacting addresses, including Thena LPs.

In [17]:
with open("../data/bsc-mpx-erc20.json", "r") as f:
    mpx_erc20_call_results = json.load(f)
len(mpx_erc20_call_results)

253

In [36]:
mpx_erc20_balances = {}
for key, value in mpx_erc20_call_results.items():
    if value > 0:
        mpx_erc20_balances[parse_address(key)] = value

df = pd.DataFrame.from_dict(mpx_erc20_balances, orient="index", columns=["balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "balance"]]
df['balance'] = df['balance'].apply(lambda x: int(x)).values.tolist()
df = df.sort_values(by='address')
df.to_csv("../products/bsc-mpx-balances.csv", index=False, float_format='%g')
sum(df['balance']) / 1e18

579031.2252789183

### fsMLP ERC-20

In [21]:
with open("../data/bsc-fsmlp.json", "r") as f:
    fsmlp_call_results = json.load(f)
len(fsmlp_call_results)

253

In [35]:
fsmlp_balances = {}
for key, value in fsmlp_call_results.items():
    if value > 0:
        fsmlp_balances[parse_address(key)] = value

df = pd.DataFrame.from_dict(fsmlp_balances, orient="index", columns=["balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "balance"]]
df['balance'] = df['balance'].apply(lambda x: int(x)).values.tolist()
df = df.sort_values(by='address')
df.to_csv("../products/bsc-fsmlp-balances.csv", index=False, float_format='%g')
sum(df['balance']) / 1e18

406866.8452357426

### Thena MPX-WBNB LP

First we obtain the MPX balance for the Thena LP.
These balances are queried from the MPX ERC-20 contract.

In [23]:
pool_mpx_total = mpx_erc20_balances[thena_mpx_wbnb_address]
pool_mpx_total / 1e18

512223.09595500864

Then we obtain the balances of the thena mpx-wbnb LP.
This represents the distribution of MPX provided for liquidity.

In [24]:
with open("../data/bsc-thena-lp.json", "r") as f:
    thena_call_results = json.load(f)
len(thena_call_results)

253

In [25]:
thena_lp_balances = {}
for key, value in thena_call_results.items():
    if value > 0:
        thena_lp_balances[parse_address(key)] = value

pool_lp_total = sum(thena_lp_balances.values())
pool_lp_total / 1e18

12257.44775264915

### Thena MPX-WBNB Gauges

Most of the LP is controlled by a gauge.
So, we must examine gauge contract balances to find the actual addresses that control the MPX.

In [26]:
with open("../data/bsc-thena-gauge.json", "r") as f:
    thena_gauge_call_results = json.load(f)
len(thena_gauge_call_results)

253

In [27]:
thena_gauge_balances = {}
for key, value in thena_gauge_call_results.items():
    if value > 0:
        thena_gauge_balances[parse_address(key)] = value

gauge_lp_total = sum(thena_gauge_balances.values())
sum(thena_gauge_balances.values()) / 1e18

5507.026316223062

### Attribute MPX to LP and Gauge

Finally, we attribute MPX to original addresses.
The MPX controlled by each address is held in proportion to the LP tokens controlled by that address.

Step 1. Examine the LP and directly assign LP to non-gauge addresses.

In [28]:
thena_attributed_lp_balances = {}
for key, value in thena_call_results.items():
    if value > 0 and parse_address(key) != thena_mpx_wbnb_gauge:
        thena_attributed_lp_balances[parse_address(key)] = value
sum(thena_attributed_lp_balances.values()) / 1e18

0.6815152704789824

Step 2. Examine Gauges and assign LP to addresses.
First determine how much LP is controlled by a gauge, then assign LP based on proportion of gauge.

In [29]:
lp_controlled_by_gauge = thena_lp_balances[thena_mpx_wbnb_gauge]
lp_controlled_by_gauge / 1e18

12256.766237378672

In [30]:
for key, value in thena_gauge_call_results.items():
    if value > 0 and parse_address(key) != thena_mpx_wbnb_gauge:
        thena_attributed_lp_balances[parse_address(key)] = (value / gauge_lp_total) * lp_controlled_by_gauge
sum(thena_attributed_lp_balances.values()) / 1e18

12257.447752649148

In [31]:
(1 - ((pool_lp_total - sum(thena_attributed_lp_balances.values())) / pool_lp_total)) * 100

99.99999999999997

We have assigned the LP.

In [34]:
df = pd.DataFrame.from_dict(thena_attributed_lp_balances, orient="index", columns=["lp_balance"])
df["address"] = df.index
df.reset_index(drop=True, inplace=True)
df = df[["address", "lp_balance"]]
df['mpx_balance'] = (df['lp_balance'] / pool_lp_total) * pool_mpx_total
df['lp_balance'] = df['lp_balance'].apply(lambda x: int(x)).values.tolist()
df['mpx_balance'] = df['mpx_balance'].apply(lambda x: int(x)).values.tolist()
df = df.sort_values(by='address')
df.to_csv("../products/bsc-thena-balances.csv", index=False, float_format='%g')
sum(df['lp_balance']) / 1e18

12257.44775264915

In [33]:
sum(df['mpx_balance']) / 1e18

512223.09595500864